In [1]:
#!pip install -q wot

Read in expression matrix, cell days, cell growth rates, and keep serum cells only

In [2]:
import wot.ot

VAR_GENE_DS_PATH = 'data/ExprMatrix.var.genes.loom'
GROWTH_SCORES_PATH = 'data/growth_gs_init.txt'
CELL_DAYS_PATH = 'data/cell_days.txt'
SERUM_CELL_IDS_PATH = 'data/serum_cell_ids.txt'

adata = wot.io.read_dataset(VAR_GENE_DS_PATH, obs=[CELL_DAYS_PATH, GROWTH_SCORES_PATH], obs_filter=SERUM_CELL_IDS_PATH)

In [3]:
adata.shape

(175472, 1479)

Initialize OT model

In [8]:
ot_model = wot.ot.OTModel(adata, tmap_out='tmaps/serum') 

Compute a single transport map

In [ ]:
tmap_adata = ot_model.compute_transport_map(7,7.5)
tmap_adata.shape

The object tmap_adata is an annotated transport matrix. 
The actual transport matrix is tmap_adata.X

tmap_adata.obs encodes the cell barcodes at time t0 and tmap_adata.var encodes the cell barcodes at time t1.

Compute another transport map from day 7.5 to 8

In [6]:
tmap_adata = ot_model.compute_transport_map(7.5, 8)

Compute all transport maps

In [7]:
#ot_model.compute_all_transport_maps()